In [ ]:
#################LAST VERSION####################

In [ ]:
"""
Lysosome + neurite segmentation + ID labeling + Napari editing

FIXED:
- TypeError: stitch_neurite_fragments_by_orientation() got an unexpected keyword argument 'cos_min'
  -> The function now accepts cos_min=... and intensity_min_frac=... (and also keeps backward-compatible
     aliases STITCH_COS_MIN / STITCH_INTENSITY_MIN_FRAC if you ever call it that way).
- Also fixed an internal bug where the function used `cos_min` but the signature previously had
  STITCH_COS_MIN, causing NameError / logic mismatch.

Run this as a single script (e.g., in a .py file).
"""

import os
import re
import numpy as np
import pandas as pd
import cv2
import imageio
import xml.etree.ElementTree as ET
from datetime import datetime

import czifile
import tifffile as tiff

from skimage.feature import blob_log
from skimage.filters import threshold_local, gaussian
from skimage.morphology import (
    remove_small_objects,
    binary_closing,
    ball,
    skeletonize_3d,
    binary_dilation,
    disk,
    binary_opening,
)
from skimage.measure import label
from skimage.draw import line_nd

try:
    from skimage.graph import route_through_array
except Exception:
    route_through_array = None

from scipy.ndimage import distance_transform_edt as _edt
from scipy.ndimage import binary_fill_holes
from scipy.ndimage import convolve
from scipy.ndimage import gaussian_filter1d
from scipy.spatial import cKDTree

import napari
import colorsys

import tkinter as tk
from tkinter import ttk, filedialog, messagebox, simpledialog

# ===============================
# USER MODE
# ===============================
NEURITE_MODE = True
EXPORT_FULLSIZE_OVERLAY = True

# Workflow B2 (manual edit via saved mask + rerun)
USE_EDITED_MASK_IF_EXISTS = True
EDITED_MASK_PATH = "neuron_mask_edited.tif"  # save this from Napari into output_dir

# Edit lysosome table in Napari (points properties) and export edited CSV
EDIT_LYSOSOME_TABLE_IN_NAPARI = True
LYSOSOME_EDITED_CSV = "lysosomes_with_cell_vs_outside_EDITED.csv"

# Attach original blob fields from lysosome_blobs_regions_ALL.csv
LYSOSOME_ALL_CSV = "lysosome_blobs_regions_ALL.csv"

# OpenCV colors are BGR
LYS_EDGE_BGR = (0, 0, 0)          # black outline
LYS_MAGENTA_BGR = (255, 0, 255)   # magenta


def attach_all_blob_fields(df_out: pd.DataFrame) -> pd.DataFrame:
    """
    Ensures df_out contains diameter_um + peak_gray taken from lysosome_blobs_regions_ALL.csv (by 'id').
    Requires df_out to have column 'id'.
    """
    if not isinstance(df_out, pd.DataFrame) or df_out.empty:
        return df_out
    if "id" not in df_out.columns:
        print("[export] WARNING: df_out has no 'id' column; cannot attach diameter_um/peak_gray from ALL.")
        return df_out
    if not os.path.isfile(LYSOSOME_ALL_CSV):
        print(f"[export] WARNING: Missing {LYSOSOME_ALL_CSV}; cannot attach diameter_um/peak_gray from ALL.")
        return df_out

    df_all = pd.read_csv(LYSOSOME_ALL_CSV)
    need = {"id", "diameter_um", "peak_gray"}
    if not need.issubset(df_all.columns):
        print(f"[export] WARNING: {LYSOSOME_ALL_CSV} missing columns {need - set(df_all.columns)}")
        return df_out

    lookup = (
        df_all[["id", "diameter_um", "peak_gray"]]
        .drop_duplicates(subset=["id"])
        .copy()
    )

    for c in ("diameter_um", "peak_gray"):
        if c in df_out.columns:
            df_out = df_out.drop(columns=[c])

    return df_out.merge(lookup, on="id", how="left")


# ===============================
# GUI
# ===============================
def get_user_config_gui(
    default_vxy_um=0.04,
    default_vz_um=None,
    default_erode_mult=1.0,
    default_blob_threshold=0.001,
    default_diam_min_um=0.0,
    default_diam_max_um="",
    default_margin_um=0.5,
    default_overlap_alpha=0.4,
    default_neighbor_max_vox=6,     # hidden
    default_viz_min_voxels=200,
    default_max_gap_um=2.0,
    default_closing_radius_vox=7,
    default_max_reasonable_vxy_um=0.5,
    default_ch1_smooth_sigma=1.0,
    default_blob_min_sigma=0.8,
    default_blob_max_sigma=3.0,
    default_blob_num_sigma=10,
    default_radial_max_radius_nm=300.0,
    default_radial_dr_nm=10.0,
    default_radial_min_drop_fraction=0.5,
    default_ch2_smooth_sigma=1.2,
    default_thresh_block_size=101,
    default_thresh_offset_std_mult=0.2,
    default_stitch_cos_min=0.7,
    default_stitch_intensity_min_frac=0.45,
    default_video_fps=8,
    default_launch_viewer=True,
    default_generate_videos=True,
):
    cfg = {"ok": False}

    root = tk.Tk()
    root.title("Lysosome + Neurite Segmentation (GUI)")
    root.resizable(False, False)

    file_var = tk.StringVar(value="")
    out_var = tk.StringVar(value="")

    erode_var = tk.StringVar(value=str(default_erode_mult))
    blob_var = tk.StringVar(value=str(default_blob_threshold))

    diam_min_var = tk.StringVar(value=str(default_diam_min_um))
    diam_max_var = tk.StringVar(value=str(default_diam_max_um))

    show_adv = tk.BooleanVar(value=False)

    margin_var = tk.StringVar(value=str(default_margin_um))
    overlap_var = tk.StringVar(value=str(default_overlap_alpha))
    vizmin_var = tk.StringVar(value=str(default_viz_min_voxels))

    maxgap_var = tk.StringVar(value=str(default_max_gap_um))
    closing_var = tk.StringVar(value=str(default_closing_radius_vox))

    # Stitching controls
    cosmin_var = tk.StringVar(value=str(default_stitch_cos_min))
    intminfrac_var = tk.StringVar(value=str(default_stitch_intensity_min_frac))

    # Local thresholding controls
    thresh_block_var = tk.StringVar(value=str(default_thresh_block_size))
    thresh_offstd_var = tk.StringVar(value=str(default_thresh_offset_std_mult))

    fps_var = tk.StringVar(value=str(default_video_fps))
    launch_viewer_var = tk.BooleanVar(value=bool(default_launch_viewer))
    gen_videos_var = tk.BooleanVar(value=bool(default_generate_videos))

    def _suggest_output_dir(fp):
        if not fp:
            return ""
        raw_dir = os.path.dirname(fp)
        raw_base = os.path.splitext(os.path.basename(fp))[0]
        stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        return os.path.join(raw_dir, f"{raw_base}_outputs_{stamp}")

    def browse_file():
        fp = filedialog.askopenfilename(
            title="Select image file",
            filetypes=[("Image files", "*.tif *.tiff *.czi"), ("All files", "*.*")],
        )
        if fp:
            file_var.set(fp)
            if not out_var.get().strip():
                out_var.set(_suggest_output_dir(fp))

    def browse_output_dir():
        d = filedialog.askdirectory(title="Select output folder")
        if d:
            fp = file_var.get().strip()
            if fp:
                raw_base = os.path.splitext(os.path.basename(fp))[0]
                stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                out_var.set(os.path.join(d, f"{raw_base}_outputs_{stamp}"))
            else:
                out_var.set(d)

    def _err(msg):
        messagebox.showerror("Invalid input", msg)
        raise ValueError(msg)

    def _float_required(s, name):
        s = (s or "").strip()
        if s == "":
            _err(f"{name} is required.")
        try:
            return float(s.replace(",", "."))
        except Exception:
            _err(f"{name} must be a number (got: {s})")

    def _float_optional(s, name):
        s = (s or "").strip()
        if s == "":
            return None
        try:
            return float(s.replace(",", "."))
        except Exception:
            _err(f"{name} must be a number (got: {s})")

    def _int_required(s, name):
        s = (s or "").strip()
        if s == "":
            _err(f"{name} is required.")
        try:
            return int(float(s.replace(",", ".")))
        except Exception:
            _err(f"{name} must be an integer (got: {s})")

    def _toggle_adv():
        if show_adv.get():
            adv_frame.grid()
        else:
            adv_frame.grid_remove()

    def run_clicked():
        fp = file_var.get().strip()
        if not fp:
            _err("Please select a file.")
        if not os.path.isfile(fp):
            _err("Selected file does not exist.")

        outd = out_var.get().strip() or _suggest_output_dir(fp)

        erode = _float_required(erode_var.get(), "ERODE_MULT")
        blobt = _float_required(blob_var.get(), "blob_log threshold")
        fps = _int_required(fps_var.get(), "video FPS")

        dmin = _float_required(diam_min_var.get(), "DIAMETER_MIN_UM (µm)")
        dmax = _float_optional(diam_max_var.get(), "DIAMETER_MAX_UM (µm)")
        if dmin < 0:
            _err("DIAMETER_MIN_UM must be >= 0.")
        if dmax is not None and dmax <= dmin:
            _err("DIAMETER_MAX_UM must be > DIAMETER_MIN_UM (or leave blank).")

        margin = _float_required(margin_var.get(), "MARGIN_UM (µm)")
        overlap = _float_required(overlap_var.get(), "OVERLAP_ALPHA (0..1)")
        vizmin = _int_required(vizmin_var.get(), "VIZ_MIN_VOXELS (voxels)")

        max_gap_um = _float_required(maxgap_var.get(), "MAX_GAP_UM (µm)")
        closing_r = _int_required(closing_var.get(), "CLOSING_RADIUS_VOX (voxels)")

        if max_gap_um <= 0:
            _err("MAX_GAP_UM must be > 0.")
        if closing_r < 1:
            _err("CLOSING_RADIUS_VOX must be >= 1.")
        if not (0.0 <= overlap <= 1.0):
            _err("OVERLAP_ALPHA must be between 0 and 1.")

        cos_min = _float_required(cosmin_var.get(), "Cos minimum")
        intensity_min_frac = _float_required(intminfrac_var.get(), "Intensity minimum fraction")

        thresh_block_size = _int_required(thresh_block_var.get(), "Block size")
        thresh_offset_std_mult = _float_required(thresh_offstd_var.get(), "Offset std multiplier")

        if not (0.0 <= cos_min <= 1.0):
            _err("Cos minimum must be between 0 and 1.")
        if not (0.0 <= intensity_min_frac <= 1.0):
            _err("Intensity minimum fraction must be between 0 and 1.")

        if thresh_block_size < 3:
            _err("Block size must be >= 3.")
        if (thresh_block_size % 2) == 0:
            _err("Block size must be an odd integer (e.g., 51, 101).")

        if thresh_offset_std_mult < 0:
            _err("Offset std multiplier must be >= 0.")

        cfg.update({
            "ok": True,
            "file_path": fp,
            "output_dir": outd,

            "ERODE_MULT": float(erode),
            "BLOB_THRESHOLD": float(blobt),

            "DEFAULT_VX_VY_UM": float(default_vxy_um),
            "DEFAULT_VZ_UM": None if default_vz_um is None else float(default_vz_um),

            "MAX_REASONABLE_VXY_UM": float(default_max_reasonable_vxy_um),

            "DIAMETER_MIN_UM": float(dmin),
            "DIAMETER_MAX_UM": None if (dmax is None) else float(dmax),

            "MARGIN_UM": float(margin),
            "OVERLAP_ALPHA": float(overlap),
            "VIZ_MIN_VOXELS": int(vizmin),

            "MAX_GAP_UM": float(max_gap_um),
            "CLOSING_RADIUS_VOX": int(closing_r),

            "NEIGHBOR_MAX_VOX": int(default_neighbor_max_vox),

            "CH1_SMOOTH_SIGMA": float(default_ch1_smooth_sigma),
            "BLOB_MIN_SIGMA": float(default_blob_min_sigma),
            "BLOB_MAX_SIGMA": float(default_blob_max_sigma),
            "BLOB_NUM_SIGMA": int(default_blob_num_sigma),

            "RADIAL_MAX_RADIUS_NM": float(default_radial_max_radius_nm),
            "RADIAL_DR_NM": float(default_radial_dr_nm),
            "RADIAL_MIN_DROP_FRACTION": float(default_radial_min_drop_fraction),

            "CH2_SMOOTH_SIGMA": float(default_ch2_smooth_sigma),

            "STITCH_COS_MIN": float(cos_min),
            "STITCH_INTENSITY_MIN_FRAC": float(intensity_min_frac),

            "THRESH_BLOCK_SIZE": int(thresh_block_size),
            "THRESH_OFFSET_STD_MULT": float(thresh_offset_std_mult),

            "VIDEO_FPS": int(fps),
            "LAUNCH_VIEWER": bool(launch_viewer_var.get()),
            "GENERATE_VIDEOS": bool(gen_videos_var.get()),
        })
        root.destroy()

    def cancel_clicked():
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", cancel_clicked)

    pad = {"padx": 10, "pady": 6}
    frm = ttk.Frame(root)
    frm.grid(row=0, column=0, sticky="nsew", **pad)

    r = 0
    ttk.Label(frm, text="Image file:").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=file_var, width=60).grid(row=r, column=1, sticky="we")
    ttk.Button(frm, text="Browse...", command=browse_file).grid(row=r, column=2, sticky="e")
    r += 1

    ttk.Label(frm, text="Output folder:").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=out_var, width=60).grid(row=r, column=1, sticky="we")
    ttk.Button(frm, text="Browse...", command=browse_output_dir).grid(row=r, column=2, sticky="e")
    r += 1

    ttk.Separator(frm).grid(row=r, column=0, columnspan=3, sticky="we", pady=8)
    r += 1

    ttk.Label(frm, text="ERODE_MULT:").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=erode_var, width=20).grid(row=r, column=1, sticky="w")
    ttk.Label(frm, text="unitless").grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Label(frm, text="blob_log threshold:").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=blob_var, width=20).grid(row=r, column=1, sticky="w")
    ttk.Label(frm, text="unitless").grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Separator(frm).grid(row=r, column=0, columnspan=3, sticky="we", pady=8)
    r += 1

    ttk.Label(frm, text="Diameter interval (µm):").grid(row=r, column=0, sticky="w")
    r += 1

    ttk.Label(frm, text="Min diameter (µm):").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=diam_min_var, width=20).grid(row=r, column=1, sticky="w")
    ttk.Label(frm, text="required").grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Label(frm, text="Max diameter (µm):").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=diam_max_var, width=20).grid(row=r, column=1, sticky="w")
    ttk.Label(frm, text="blank = no max").grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Separator(frm).grid(row=r, column=0, columnspan=3, sticky="we", pady=8)
    r += 1

    ttk.Checkbutton(frm, text="Launch Napari viewer", variable=launch_viewer_var)\
        .grid(row=r, column=0, columnspan=2, sticky="w")
    ttk.Checkbutton(frm, text="Generate videos", variable=gen_videos_var)\
        .grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Label(frm, text="Video FPS:").grid(row=r, column=0, sticky="w")
    ttk.Entry(frm, textvariable=fps_var, width=20).grid(row=r, column=1, sticky="w")
    ttk.Label(frm, text="frames/sec").grid(row=r, column=2, sticky="w")
    r += 1

    ttk.Separator(frm).grid(row=r, column=0, columnspan=3, sticky="we", pady=8)
    r += 1

    ttk.Checkbutton(frm, text="Show advanced settings", variable=show_adv, command=_toggle_adv)\
        .grid(row=r, column=0, columnspan=3, sticky="w")
    r += 1

    adv_frame = ttk.LabelFrame(frm, text="Advanced")
    adv_frame.grid(row=r, column=0, columnspan=3, sticky="we", pady=6)
    adv_frame.grid_remove()

    def add_row(parent, label_txt, var, unit_txt):
        row = parent.grid_size()[1]  # next row index
        ttk.Label(parent, text=label_txt).grid(row=row, column=0, sticky="w", padx=8, pady=3)
        ttk.Entry(parent, textvariable=var, width=18).grid(row=row, column=1, sticky="w", padx=8, pady=3)
        ttk.Label(parent, text=unit_txt).grid(row=row, column=2, sticky="w", padx=8, pady=3)

    add_row(adv_frame, "MARGIN_UM:", margin_var, "µm")
    add_row(adv_frame, "OVERLAP_ALPHA:", overlap_var, "unitless (0–1)")
    add_row(adv_frame, "VIZ_MIN_VOXELS:", vizmin_var, "voxels")
    add_row(adv_frame, "MAX_GAP_UM:", maxgap_var, "µm")
    add_row(adv_frame, "CLOSING_RADIUS_VOX:", closing_var, "voxels")

    stitching_frame = ttk.LabelFrame(adv_frame, text="Stitching")
    stitching_frame.grid(row=adv_frame.grid_size()[1], column=0, columnspan=3, sticky="we", padx=6, pady=(10, 6))
    add_row(stitching_frame, "Cos minimum:", cosmin_var, "unitless (0–1)")
    add_row(stitching_frame, "Intensity minimum fraction:", intminfrac_var, "fraction (0–1)")

    thresh_frame = ttk.LabelFrame(adv_frame, text="Local Thresholding")
    thresh_frame.grid(row=adv_frame.grid_size()[1], column=0, columnspan=3, sticky="we", padx=6, pady=(10, 6))
    add_row(thresh_frame, "Block size:", thresh_block_var, "pixels (odd integer)")
    add_row(thresh_frame, "Offset std multiplier:", thresh_offstd_var, "×STD (unitless)")

    r += 1
    ttk.Separator(frm).grid(row=r, column=0, columnspan=3, sticky="we", pady=8)
    r += 1

    btns = ttk.Frame(frm)
    btns.grid(row=r, column=0, columnspan=3, sticky="e")
    ttk.Button(btns, text="Cancel", command=cancel_clicked).grid(row=0, column=0, padx=6)
    ttk.Button(btns, text="Run", command=run_clicked).grid(row=0, column=1, padx=6)

    root.mainloop()
    if not cfg.get("ok"):
        raise SystemExit("Cancelled.")
    return cfg


# ===============================
# Metadata parsing
# ===============================
def _parse_ome_xml(xml_text):
    if not xml_text:
        return None, None, None

    def grab(attr):
        m = re.search(fr'PhysicalSize{attr}="([\d\.eE+-]+)"', xml_text)
        return float(m.group(1)) if m else None

    return grab("X"), grab("Y"), grab("Z")


def _parse_czi_scaling(czi_text):
    if not czi_text:
        return None, None, None
    if isinstance(czi_text, (bytes, bytearray)):
        czi_text = czi_text.decode("utf-8", errors="ignore")
    czi_text = czi_text.replace("\x00", "")

    def _to_float(s):
        if s is None:
            return None
        try:
            return float(str(s).strip().replace(",", "."))
        except Exception:
            return None

    def _to_um(val, unit_hint=None):
        if val is None:
            return None
        if unit_hint:
            u = str(unit_hint).strip().lower()
            if u in ("m", "meter", "metre", "meters", "metres"):
                return val * 1e6
            if u in ("µm", "um", "micron", "microns", "micrometer", "micrometre"):
                return val
            if u in ("nm", "nanometer", "nanometre", "nanometers", "nanometres"):
                return val / 1000.0

        if val < 1e-3:
            return val * 1e6
        if val < 10:
            return val
        if val < 1e5:
            return val / 1000.0
        return None

    try:
        root = ET.fromstring(czi_text)
    except Exception:
        def _grab(axis):
            mm = re.search(
                rf'<Distance[^>]*Id="{axis}"[^>]*>.*?<Value>\s*([0-9eE\+\-\.]+)\s*</Value>',
                czi_text, flags=re.IGNORECASE | re.DOTALL,
            )
            return _to_float(mm.group(1)) if mm else None

        return _to_um(_grab("X")), _to_um(_grab("Y")), _to_um(_grab("Z"))

    sx = sy = sz = None
    for d in root.findall(".//{*}Distance"):
        axis = d.attrib.get("Id") or d.attrib.get("id") or d.attrib.get("Axis") or d.attrib.get("axis")
        if not axis:
            continue
        axis = axis.upper()
        unit = d.attrib.get("Unit") or d.attrib.get("unit")

        valf = _to_float(d.attrib.get("Value") or d.attrib.get("value"))
        if valf is None:
            v_el = d.find(".//{*}Value")
            if v_el is not None and v_el.text:
                valf = _to_float(v_el.text)

        if valf is None:
            for child in d.iter():
                if child is d:
                    continue
                if str(child.tag).lower().endswith("value"):
                    valf = _to_float(child.attrib.get("Value") or child.attrib.get("value")) or _to_float(child.text)
                    if valf is not None:
                        break

        val_um = _to_um(valf, unit_hint=unit)
        if val_um is None:
            continue

        if axis == "X":
            sx = val_um
        elif axis == "Y":
            sy = val_um
        elif axis == "Z":
            sz = val_um

    return sx, sy, sz


def load_any(file_path):
    ext = os.path.splitext(file_path)[1].lower()

    if ext in (".tif", ".tiff"):
        with tiff.TiffFile(file_path) as tf:
            arr = tf.asarray()
            try:
                ome_xml = tf.ome_metadata
            except Exception:
                ome_xml = None
            vx_um = vy_um = vz_um = None
            if ome_xml:
                vx_um, vy_um, vz_um = _parse_ome_xml(ome_xml)

        img = np.squeeze(arr)
        if img.ndim == 4:
            if img.shape[0] == 2:
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError("Unexpected TIFF shape for 2 channels")
        else:
            raise RuntimeError("Unexpected TIFF shape")
        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "tiff"}

    if ext == ".czi":
        with czifile.CziFile(file_path) as cf:
            arr = cf.asarray()
            try:
                czi_xml = cf.metadata()
            except Exception:
                czi_xml = None

        vx_um = vy_um = vz_um = None
        if czi_xml:
            vx_um, vy_um, vz_um = _parse_czi_scaling(czi_xml)

        img = np.squeeze(arr)
        if img.ndim == 4:
            if img.shape[0] == 2:
                ch1, ch2 = img[0], img[1]
            elif img.shape[1] == 2:
                ch1, ch2 = img[:, 0], img[:, 1]
            elif img.shape[-1] == 2:
                ch1, ch2 = img[..., 0], img[..., 1]
            else:
                raise RuntimeError("Unexpected CZI shape for 2 channels")
        else:
            raise RuntimeError("Unexpected CZI shape")

        return ch1, ch2, (vx_um, vy_um, vz_um), {"type": "czi"}

    raise ValueError("Unsupported file format")


def _prompt_missing_size(title, prompt, default_value):
    root = tk.Tk()
    root.withdraw()
    try:
        root.attributes("-topmost", True)
    except Exception:
        pass

    use_default = messagebox.askyesno(title, f"{prompt}\n\nUse default value: {default_value} ?")
    if use_default:
        root.destroy()
        return float(default_value)

    val = simpledialog.askfloat(
        title,
        "Enter a new value:",
        initialvalue=float(default_value),
        minvalue=1e-12
    )
    root.destroy()
    if val is None:
        raise SystemExit("Cancelled.")
    return float(val)


# ===============================
# Morphology: ellipsoid footprint (anisotropic)
# ===============================
def ellipsoid_footprint_from_radius_um(radius_um, vz_um, vy_um, vx_um, min_r_vox=1):
    """
    Build a 3D ellipsoidal footprint whose semi-axes correspond to `radius_um`
    in physical space, respecting anisotropic voxel spacing.
    returns: bool array footprint (Z,Y,X)
    """
    radius_um = float(radius_um)
    if radius_um <= 0:
        return np.ones((1, 1, 1), dtype=bool)

    rz = int(np.ceil(radius_um / max(float(vz_um), 1e-12)))
    ry = int(np.ceil(radius_um / max(float(vy_um), 1e-12)))
    rx = int(np.ceil(radius_um / max(float(vx_um), 1e-12)))

    rz = max(min_r_vox, rz)
    ry = max(min_r_vox, ry)
    rx = max(min_r_vox, rx)

    z = np.arange(-rz, rz + 1, dtype=np.float32)[:, None, None]
    y = np.arange(-ry, ry + 1, dtype=np.float32)[None, :, None]
    x = np.arange(-rx, rx + 1, dtype=np.float32)[None, None, :]

    zz = (z / rz) ** 2
    yy = (y / ry) ** 2
    xx = (x / rx) ** 2
    return (zz + yy + xx) <= 1.0


# ===============================
# Option B: direction-aware stitching + intensity-aware + shortest-path option
# ===============================
def stitch_neurite_fragments_by_orientation(
    neuron_mask,
    vx_um, vy_um, vz_um,
    max_gap_um=7.0,
    cos_min=0.7,  # <-- FIX: accept cos_min
    bridge_dilate_radius_vox=3,
    intensity_img=None,                 # pass ch2 (normalized+smoothed)
    intensity_bg_quantile=0.995,
    intensity_fg_quantile=0.50,
    intensity_mix=0.20,
    intensity_min_frac=0.45,            # <-- FIX: accept intensity_min_frac
    use_shortest_path=True,
    sp_pad_um=2.0,
    sp_max_box_dim_vox=400,
    sp_dark_penalty=1.0,
    sp_max_path_len_factor=3.5,
    # backward-compatible aliases (ignored unless passed explicitly by name):
    STITCH_COS_MIN=None,
    STITCH_INTENSITY_MIN_FRAC=None,
):
    """
    Stitch neurite fragments by pairing endpoints whose local directions align
    and whose gap is <= max_gap_um. Optionally validates bridges using intensity.

    Backward-compatibility:
      - If STITCH_COS_MIN is provided, it overrides cos_min
      - If STITCH_INTENSITY_MIN_FRAC is provided, it overrides intensity_min_frac
    """
    if STITCH_COS_MIN is not None:
        cos_min = float(STITCH_COS_MIN)
    if STITCH_INTENSITY_MIN_FRAC is not None:
        intensity_min_frac = float(STITCH_INTENSITY_MIN_FRAC)

    if neuron_mask is None or neuron_mask.sum() == 0:
        return neuron_mask

    skel = skeletonize_3d(neuron_mask).astype(bool)
    if skel.sum() == 0:
        return neuron_mask

    thr_int = None
    if intensity_img is not None:
        try:
            fg = intensity_img[neuron_mask]
            bg = intensity_img[~neuron_mask]
            bg_hi = float(np.quantile(bg, intensity_bg_quantile)) if bg.size else 0.0
            fg_q = float(np.quantile(fg, intensity_fg_quantile)) if fg.size else max(bg_hi, 1e-6)
            thr_int = bg_hi + float(intensity_mix) * (fg_q - bg_hi)
        except Exception:
            thr_int = None

    kernel = np.ones((3, 3, 3), dtype=np.uint8)
    kernel[1, 1, 1] = 0
    neigh = convolve(skel.astype(np.uint8), kernel, mode="constant", cval=0)

    endpoints = skel & (neigh == 1)
    pts = np.argwhere(endpoints)  # zyx
    n = pts.shape[0]
    if n < 2:
        return neuron_mask

    spacing = np.array([vz_um, vy_um, vx_um], dtype=np.float32)

    dirs = np.zeros((n, 3), dtype=np.float32)
    Z, Y, X = skel.shape

    for i, (z, y, x) in enumerate(pts):
        z1, z2 = max(0, z - 1), min(Z, z + 2)
        y1, y2 = max(0, y - 1), min(Y, y + 2)
        x1, x2 = max(0, x - 1), min(X, x + 2)

        patch = skel[z1:z2, y1:y2, x1:x2]
        nbrs = np.argwhere(patch)
        nbrs = nbrs + np.array([z1, y1, x1], dtype=np.int32)
        nbrs = nbrs[~((nbrs[:, 0] == z) & (nbrs[:, 1] == y) & (nbrs[:, 2] == x))]
        if nbrs.shape[0] == 0:
            continue

        d2 = np.sum((nbrs - np.array([z, y, x], dtype=np.int32)) ** 2, axis=1)
        nb = nbrs[int(np.argmin(d2))]

        v_vox = (np.array([z, y, x], dtype=np.float32) - nb.astype(np.float32))
        v_um = v_vox * spacing
        norm = float(np.linalg.norm(v_um))
        if norm > 0:
            dirs[i] = v_um / norm

    def _path_len_um(coords_zyx: np.ndarray) -> float:
        if coords_zyx.shape[0] < 2:
            return 0.0
        d = coords_zyx[1:] - coords_zyx[:-1]
        d_um = d.astype(np.float32) * spacing[None, :]
        step = np.sqrt(np.sum(d_um * d_um, axis=1))
        return float(np.sum(step))

    def _shortest_path_coords(p1_zyx, p2_zyx):
        if (not use_shortest_path) or (intensity_img is None) or (thr_int is None) or (route_through_array is None):
            return None

        pad_vox = np.ceil(np.array([sp_pad_um / vz_um, sp_pad_um / vy_um, sp_pad_um / vx_um])).astype(int)
        lo = np.minimum(p1_zyx, p2_zyx) - pad_vox
        hi = np.maximum(p1_zyx, p2_zyx) + pad_vox + 1

        lo = np.maximum(lo, 0)
        hi = np.minimum(hi, np.array([Z, Y, X], dtype=int))

        box_shape = hi - lo
        if int(box_shape.max()) > int(sp_max_box_dim_vox):
            return None

        subI = intensity_img[lo[0]:hi[0], lo[1]:hi[1], lo[2]:hi[2]].astype(np.float32)
        subI = np.clip(subI, 0.0, 1.0)

        cost = (1.0 - subI)
        dark = subI < float(thr_int)
        cost = cost + dark.astype(np.float32) * float(sp_dark_penalty)
        cost = cost + 1e-3

        start = tuple((p1_zyx - lo).astype(int))
        end = tuple((p2_zyx - lo).astype(int))

        try:
            idxs, _total_cost = route_through_array(cost, start, end, fully_connected=True, geometric=True)
        except Exception:
            return None

        coords = np.array(idxs, dtype=np.int32) + lo[None, :]

        max_len = float(max_gap_um) * float(sp_max_path_len_factor)
        if _path_len_um(coords) > max_len:
            return None

        vals = intensity_img[tuple(coords.T)]
        if vals.size > 2:
            vals = vals[1:-1]
        if vals.size == 0:
            return None

        frac = float(np.mean(vals >= float(thr_int)))
        if frac < float(intensity_min_frac):
            return None

        return coords

    candidates = []
    for i in range(n):
        if not np.isfinite(dirs[i]).all() or np.linalg.norm(dirs[i]) == 0:
            continue
        for j in range(i + 1, n):
            if not np.isfinite(dirs[j]).all() or np.linalg.norm(dirs[j]) == 0:
                continue

            d_vox = (pts[j] - pts[i]).astype(np.float32)
            d_um_vec = d_vox * spacing
            dist = float(np.linalg.norm(d_um_vec))
            if dist <= 0 or dist > max_gap_um:
                continue

            v = d_um_vec / dist
            if float(np.dot(dirs[i], v)) < float(cos_min):
                continue
            if float(np.dot(dirs[j], -v)) < float(cos_min):
                continue

            candidates.append((dist, i, j))

    if not candidates:
        return neuron_mask

    candidates.sort(key=lambda t: t[0])

    used = np.zeros(n, dtype=bool)
    bridge = np.zeros_like(neuron_mask, dtype=bool)

    for dist, i, j in candidates:
        if used[i] or used[j]:
            continue

        p1 = pts[i].astype(np.int32)
        p2 = pts[j].astype(np.int32)

        coords = _shortest_path_coords(p1, p2)

        if coords is None:
            rr = line_nd(tuple(p1), tuple(p2), endpoint=True)
            if intensity_img is not None and thr_int is not None:
                vals = intensity_img[rr]
                if vals.size > 2:
                    vals = vals[1:-1]
                if vals.size > 0:
                    frac = float(np.mean(vals >= float(thr_int)))
                    if frac < float(intensity_min_frac):
                        continue
            bridge[rr] = True
        else:
            bridge[tuple(coords.T)] = True

        used[i] = True
        used[j] = True

    if not bridge.any():
        return neuron_mask

    bridged = neuron_mask | binary_dilation(bridge, ball(int(bridge_dilate_radius_vox)))
    bridged = binary_closing(bridged, ball(1))
    return bridged


# ===============================
# Helper: refine radii by distance transform
# ===============================
def refine_radii_via_dt(img3d, blobs, win_px=40, bin_method="local"):
    if blobs is None or len(blobs) == 0:
        return blobs

    Z, H, W = img3d.shape
    out = blobs.copy().astype(np.float32)

    for i, (zc, yc, xc, _) in enumerate(out):
        z, y, x = int(round(zc)), int(round(yc)), int(round(xc))
        if not (0 <= z < Z and 0 <= y < H and 0 <= x < W):
            continue

        y1, y2 = max(0, y - win_px), min(H, y + win_px + 1)
        x1, x2 = max(0, x - win_px), min(W, x + win_px + 1)
        patch = img3d[z, y1:y2, x1:x2]

        ws = max(11, 2 * (win_px // 2) + 1)
        thr = threshold_local(patch, block_size=ws, offset=-0.4 * np.std(patch))
        bw = patch > thr

        bw = binary_opening(bw, footprint=disk(1))
        bw = remove_small_objects(bw, min_size=3, connectivity=2)

        yy, xx = y - y1, x - x1
        if not (0 <= yy < bw.shape[0] and 0 <= xx < bw.shape[1]) or not bw[yy, xx]:
            continue

        lab = label(bw)
        lbl = lab[yy, xx]
        if lbl == 0:
            continue
        bw_obj = (lab == lbl)
        dt = _edt(bw_obj)
        r_px = float(dt[yy, xx])
        if r_px > 0:
            out[i, 3] = r_px

    return out


def refine_radii_via_radial_intensity(
    img3d,
    blobs,
    vx_um,
    vy_um,
    vz_um,
    max_radius_nm=300.0,
    dr_nm=10.0,
    min_drop_fraction=0.5,
):
    if blobs is None or len(blobs) == 0:
        return blobs

    img = img3d.astype(np.float32)
    Z, Y, X = img.shape
    blobs_out = blobs.copy().astype(np.float32)

    max_r_um = max_radius_nm / 1000.0
    dr_um = dr_nm / 1000.0

    r_edges = np.arange(0.0, max_r_um + dr_um, dr_um)
    if r_edges.size < 2:
        return blobs_out
    r_centers = 0.5 * (r_edges[:-1] + r_edges[1:])

    px_um_xy = float(np.sqrt(vx_um * vy_um))

    for i, (zc, yc, xc, r_px_init) in enumerate(blobs_out):
        z0 = int(round(zc))
        y0 = int(round(yc))
        x0 = int(round(xc))

        if not (0 <= z0 < Z and 0 <= y0 < Y and 0 <= x0 < X):
            continue

        rz = max(1, int(np.ceil(max_r_um / vz_um)))
        ry = max(1, int(np.ceil(max_r_um / vy_um)))
        rx = max(1, int(np.ceil(max_r_um / vx_um)))

        z1, z2 = max(0, z0 - rz), min(Z, z0 + rz + 1)
        y1, y2 = max(0, y0 - ry), min(Y, y0 + ry + 1)
        x1, x2 = max(0, x0 - rx), min(X, x0 + rx + 1)
        if z1 >= z2 or y1 >= y2 or x1 >= x2:
            continue

        patch = img[z1:z2, y1:y2, x1:x2]

        zz, yy, xx = np.mgrid[z1:z2, y1:y2, x1:x2]
        dz_um = (zz - z0) * vz_um
        dy_um = (yy - y0) * vy_um
        dx_um = (xx - x0) * vx_um
        r_um = np.sqrt(dz_um**2 + dy_um**2 + dx_um**2)

        mask = (r_um <= max_r_um)
        if not np.any(mask):
            continue

        r_vals = r_um[mask].ravel()
        I_vals = patch[mask].ravel()

        bin_idx = np.digitize(r_vals, r_edges) - 1
        valid = (bin_idx >= 0) & (bin_idx < r_centers.size)
        if not np.any(valid):
            continue

        bin_idx = bin_idx[valid]
        I_vals = I_vals[valid]

        sums = np.bincount(bin_idx, weights=I_vals, minlength=r_centers.size)
        counts = np.bincount(bin_idx, minlength=r_centers.size)
        with np.errstate(invalid="ignore", divide="ignore"):
            prof = sums / np.maximum(counts, 1)

        have = counts > 0
        if not np.any(have):
            continue

        I_prof = prof[have].astype(np.float32)
        I_smooth = gaussian_filter1d(I_prof, sigma=1.0)

        I_max = float(I_smooth.max())
        I_min = float(I_smooth.min())
        if I_max <= 0:
            continue
        if (I_max - I_min) / max(I_max, 1e-9) < float(min_drop_fraction):
            continue

        I_half = I_min + 0.5 * (I_max - I_min)

        peak_idx = int(np.argmax(I_smooth))
        n_bins = len(I_smooth)

        left_idx = peak_idx
        while left_idx > 0 and I_smooth[left_idx] >= I_half:
            left_idx -= 1
        if left_idx < peak_idx and I_smooth[left_idx] < I_half:
            left_idx += 1

        right_idx = peak_idx
        while right_idx < n_bins - 1 and I_smooth[right_idx] >= I_half:
            right_idx += 1
        if right_idx > peak_idx and I_smooth[right_idx] < I_half:
            right_idx -= 1

        if right_idx <= left_idx:
            continue

        r_prof = r_centers[have]
        radius_um = 0.5 * (float(r_prof[right_idx]) - float(r_prof[left_idx]))
        if radius_um <= 0:
            continue

        r_fwhm_px = radius_um / max(px_um_xy, 1e-9)
        blobs_out[i, 3] = max(float(r_px_init), float(r_fwhm_px))

    return blobs_out


# ===============================
# Distance/overlap helpers
# ===============================
def nearest_cell_label(cell_seg, z, y, x, max_r=12):
    Z, Y, X = cell_seg.shape
    for r in range(1, max_r + 1):
        z1, z2 = max(0, z - r), min(Z, z + r + 1)
        y1, y2 = max(0, y - r), min(Y, y + r + 1)
        x1, x2 = max(0, x - r), min(X, x + r + 1)
        patch = cell_seg[z1:z2, y1:y2, x1:x2]
        lab = patch[patch > 0]
        if lab.size:
            return int(np.bincount(lab.ravel()).argmax())
    return 0


def sphere_overlap_fraction(zc_um, yc_um, xc_um, r_um, mask_bool, vx_um, vy_um, vz_um):
    if r_um <= 0:
        return 0.0

    zc = int(round(zc_um / vz_um))
    yc = int(round(yc_um / vy_um))
    xc = int(round(xc_um / vx_um))

    rz = max(1, int(np.ceil(r_um / vz_um)))
    ry = max(1, int(np.ceil(r_um / vy_um)))
    rx = max(1, int(np.ceil(r_um / vx_um)))

    Z, Y, X = mask_bool.shape
    z1, z2 = max(0, zc - rz), min(Z, zc + rz + 1)
    y1, y2 = max(0, yc - ry), min(Y, yc + ry + 1)
    x1, x2 = max(0, xc - rx), min(X, xc + rx + 1)
    if z1 >= z2 or y1 >= y2 or x1 >= x2:
        return 0.0

    zz, yy, xx = np.mgrid[z1:z2, y1:y2, x1:x2]
    dz = (zz - zc) * vz_um
    dy = (yy - yc) * vy_um
    dx = (xx - xc) * vx_um
    sphere = (dz*dz + dy*dy + dx*dx) <= (r_um * r_um)

    if not np.any(sphere):
        return 0.0

    in_mask = mask_bool[z1:z2, y1:y2, x1:x2] & sphere
    return float(in_mask.sum()) / float(sphere.sum())


# ===============================
# Export helpers
# ===============================
def _norm_u8_stack(vol):
    vmin, vmax = float(vol.min()), float(vol.max())
    if vmax > vmin:
        out = (np.clip((vol - vmin) / (vmax - vmin), 0, 1) * 255.0).astype(np.uint8)
    else:
        out = np.zeros_like(vol, dtype=np.uint8)
    return out


def make_label_colormap(n_labels, seed_hue=0.13):
    colors = np.zeros((n_labels + 1, 3), dtype=np.uint8)
    if n_labels <= 0:
        return colors
    for i in range(1, n_labels + 1):
        h = (seed_hue + (i - 1) / max(n_labels, 1)) % 1.0
        s = 1.0
        v = 1.0
        r, g, b = colorsys.hsv_to_rgb(h, s, v)
        colors[i] = (int(255 * r), int(255 * g), int(255 * b))
    return colors


def export_fullsize_overlay_stack(
    img_ch1,
    img_ch2_raw,
    cell_seg_viz,
    df,
    vx_um, vy_um, vz_um,
    output_dir,
    alpha_labels=0.45,
    draw_only_inside=True,
    fps=8,
    basename="FULLSIZE_overlay_ID_Lysosomes_MAGENTA",
):
    os.makedirs(output_dir, exist_ok=True)

    ch1_u8 = _norm_u8_stack(img_ch1.astype(np.float32))
    ch2_u8 = _norm_u8_stack(img_ch2_raw.astype(np.float32))

    Z, H, W = ch2_u8.shape
    n_labels = int(cell_seg_viz.max()) if isinstance(cell_seg_viz, np.ndarray) else 0
    cmap = make_label_colormap(n_labels, seed_hue=0.13)

    use_df = None
    if isinstance(df, pd.DataFrame) and len(df) > 0 and {"z_um", "y_um", "x_um", "radius_um"}.issubset(df.columns):
        if draw_only_inside and "location_ch2" in df.columns:
            use_df = df[df["location_ch2"] == "cell"].copy()
        else:
            use_df = df.copy()
        use_df = use_df[
            np.isfinite(use_df["z_um"]) &
            np.isfinite(use_df["y_um"]) &
            np.isfinite(use_df["x_um"]) &
            np.isfinite(use_df["radius_um"])
        ].copy()

    px_um_xy = float(np.sqrt(vx_um * vy_um))
    frames = np.zeros((Z, H, W, 3), dtype=np.uint8)

    for z in range(Z):
        base = np.dstack([ch1_u8[z], ch2_u8[z], ch1_u8[z]]).astype(np.float32)

        lab2d = cell_seg_viz[z].astype(np.int32)
        lab_rgb = cmap[lab2d].astype(np.float32)

        mask = (lab2d > 0)[..., None].astype(np.float32)
        out = base * (1.0 - alpha_labels * mask) + lab_rgb * (alpha_labels * mask)

        if use_df is not None and len(use_df) > 0:
            zc = (use_df["z_um"].to_numpy() / vz_um).astype(float)
            yc = (use_df["y_um"].to_numpy() / vy_um).astype(float)
            xc = (use_df["x_um"].to_numpy() / vx_um).astype(float)
            r_um = use_df["radius_um"].to_numpy().astype(float)

            dz_um = np.abs(zc - z) * vz_um
            hits = dz_um <= r_um
            if np.any(hits):
                r_proj_um = np.sqrt(np.clip(r_um[hits]**2 - dz_um[hits]**2, 0.0, None))
                r_proj_px = r_proj_um / max(px_um_xy, 1e-12)
                ys = np.rint(yc[hits]).astype(int)
                xs = np.rint(xc[hits]).astype(int)

                out_u8 = np.clip(out, 0, 255).astype(np.uint8)
                for y, x, rp in zip(ys, xs, r_proj_px):
                    rr = int(max(3, round(rp)))
                    if 0 <= y < H and 0 <= x < W and rr > 0:
                        cv2.circle(out_u8, (x, y), rr, LYS_EDGE_BGR, 4, lineType=cv2.LINE_AA)
                        cv2.circle(out_u8, (x, y), rr, LYS_MAGENTA_BGR, 2, lineType=cv2.LINE_AA)
                        cv2.circle(out_u8, (x, y), 1,  LYS_MAGENTA_BGR, -1, lineType=cv2.LINE_AA)
                out = out_u8.astype(np.float32)

        frames[z] = np.clip(out, 0, 255).astype(np.uint8)

    tiff_path = os.path.join(output_dir, f"{basename}.tif")
    tiff.imwrite(tiff_path, frames, photometric="rgb")
    print("Saved full-size RGB TIFF stack:", tiff_path)

    mp4_path = os.path.join(output_dir, f"{basename}.mp4")
    try:
        with imageio.get_writer(
            mp4_path, fps=int(fps), format="FFMPEG", codec="libx264", macro_block_size=None
        ) as w:
            for fr in frames:
                w.append_data(fr)
        print("Saved full-size MP4:", mp4_path)
    except Exception as e:
        gif_path = os.path.join(output_dir, f"{basename}.gif")
        imageio.mimsave(gif_path, list(frames), fps=int(fps))
        print("FFMPEG failed, saved GIF instead:", gif_path, "Error:", e)

    return tiff_path, mp4_path


# ===============================
# MAIN
# ===============================
DEFAULT_VX_VY_UM = 0.04
DEFAULT_VZ_UM = None

cfg = get_user_config_gui(
    default_vxy_um=DEFAULT_VX_VY_UM,
    default_vz_um=DEFAULT_VZ_UM,
    default_erode_mult=1.0,
    default_blob_threshold=0.001,
    default_diam_min_um=0.0,
    default_diam_max_um="",
)

file_path = cfg["file_path"]
output_dir = cfg["output_dir"]
os.makedirs(output_dir, exist_ok=True)
os.chdir(output_dir)

print("Selected file:", file_path)
print("Outputs will be saved to:", output_dir)

ERODE_MULT = cfg["ERODE_MULT"]
BLOB_THRESHOLD = cfg["BLOB_THRESHOLD"]

MAX_REASONABLE_VXY_UM = cfg["MAX_REASONABLE_VXY_UM"]
MARGIN_UM = cfg["MARGIN_UM"]
OVERLAP_ALPHA = cfg["OVERLAP_ALPHA"]
NEIGHBOR_MAX_VOX = cfg["NEIGHBOR_MAX_VOX"]
VIZ_MIN_VOXELS = cfg["VIZ_MIN_VOXELS"]

MAX_GAP_UM = cfg["MAX_GAP_UM"]
CLOSING_RADIUS_VOX = cfg["CLOSING_RADIUS_VOX"]

CH1_SMOOTH_SIGMA = cfg["CH1_SMOOTH_SIGMA"]
BLOB_MIN_SIGMA = cfg["BLOB_MIN_SIGMA"]
BLOB_MAX_SIGMA = cfg["BLOB_MAX_SIGMA"]
BLOB_NUM_SIGMA = cfg["BLOB_NUM_SIGMA"]

RADIAL_MAX_RADIUS_NM = cfg["RADIAL_MAX_RADIUS_NM"]
RADIAL_DR_NM = cfg["RADIAL_DR_NM"]
RADIAL_MIN_DROP_FRACTION = cfg["RADIAL_MIN_DROP_FRACTION"]

CH2_SMOOTH_SIGMA = cfg["CH2_SMOOTH_SIGMA"]
THRESH_BLOCK_SIZE = cfg["THRESH_BLOCK_SIZE"]
THRESH_OFFSET_STD_MULT = cfg["THRESH_OFFSET_STD_MULT"]

FPS = cfg["VIDEO_FPS"]
LAUNCH_VIEWER = cfg["LAUNCH_VIEWER"]
GENERATE_VIDEOS = cfg["GENERATE_VIDEOS"]

DIAMETER_MIN_UM = float(cfg.get("DIAMETER_MIN_UM", 0.0))
DIAMETER_MAX_UM = cfg.get("DIAMETER_MAX_UM", None)

img_ch1, img_ch2, (vx_um, vy_um, vz_um), meta = load_any(file_path)
print(f"[metadata] vx_um={vx_um}  vy_um={vy_um}  vz_um={vz_um}")

if vx_um is None or vy_um is None:
    vx_um = vy_um = _prompt_missing_size(
        "Missing metadata",
        "XY pixel size metadata is missing (µm/px).",
        float(cfg["DEFAULT_VX_VY_UM"]),
    )

if vz_um is None:
    z_default = float(cfg["DEFAULT_VZ_UM"]) if (cfg["DEFAULT_VZ_UM"] is not None) else float(vx_um)
    vz_um = _prompt_missing_size(
        "Missing metadata",
        "Z step metadata is missing (µm/slice).",
        z_default,
    )

if vx_um > MAX_REASONABLE_VXY_UM:
    raise ValueError(f"XY pixel size too large: {vx_um} µm/px")

px_um_xy = float(np.sqrt(vx_um * vy_um))
px_um = px_um_xy * 0.7
voxel_um3 = vx_um * vy_um * vz_um
print(f"Voxel size (µm): X={vx_um}, Y={vy_um}, Z={vz_um}")

# ==========================================
# Lysosome detection (Ch1)
# ==========================================
image = img_ch1
image_smooth = gaussian(image, sigma=CH1_SMOOTH_SIGMA)

blobs = blob_log(
    image_smooth,
    min_sigma=BLOB_MIN_SIGMA,
    max_sigma=BLOB_MAX_SIGMA,
    num_sigma=BLOB_NUM_SIGMA,
    threshold=BLOB_THRESHOLD
)

if len(blobs) > 0:
    blobs[:, 3] *= np.sqrt(3)
    blobs = refine_radii_via_dt(image_smooth, blobs)
    blobs = refine_radii_via_radial_intensity(
        image_smooth, blobs, vx_um, vy_um, vz_um,
        max_radius_nm=RADIAL_MAX_RADIUS_NM,
        dr_nm=RADIAL_DR_NM,
        min_drop_fraction=RADIAL_MIN_DROP_FRACTION
    )

peak_gray = np.zeros(len(blobs), dtype=np.uint16)
Z0, Y0, X0 = image.shape
rad = 1
for i, (zc, yc, xc, _) in enumerate(blobs):
    zc_i = int(round(zc))
    yc_i = int(round(yc))
    xc_i = int(round(xc))

    z1, z2 = max(0, zc_i - rad), min(Z0, zc_i + rad + 1)
    y1, y2 = max(0, yc_i - rad), min(Y0, yc_i + rad + 1)
    x1, x2 = max(0, xc_i - rad), min(X0, xc_i + rad + 1)

    peak_gray[i] = np.max(image[z1:z2, y1:y2, x1:x2]).astype(np.uint16)

if len(blobs) > 0:
    z_um = blobs[:, 0] * vz_um
    y_um = blobs[:, 1] * vy_um
    x_um = blobs[:, 2] * vx_um
    radius_um = blobs[:, 3] * px_um
    diameter_um = 2 * radius_um
    volume_um3 = (4/3) * np.pi * radius_um**3
    blob_ids = np.arange(1, len(blobs) + 1, dtype=int)
else:
    z_um = y_um = x_um = radius_um = diameter_um = volume_um3 = np.array([])
    blob_ids = np.array([], dtype=int)

df_all = pd.DataFrame({
    "id": blob_ids,
    "z_um": z_um,
    "y_um": y_um,
    "x_um": x_um,
    "radius_um": radius_um,
    "diameter_um": diameter_um,
    "volume_um3": volume_um3,
    "peak_gray": peak_gray,
})

dmax_eff = np.inf if (DIAMETER_MAX_UM is None) else float(DIAMETER_MAX_UM)
if len(df_all) > 0:
    dd = df_all["diameter_um"].to_numpy(dtype=float)
    keep = np.isfinite(dd) & (dd >= float(DIAMETER_MIN_UM)) & (dd <= dmax_eff)
    df = df_all.loc[keep].copy()
else:
    df = df_all.copy()

df_all.to_csv("lysosome_blobs_regions_ALL.csv", index=False)
df.to_csv("lysosome_blobs_regions.csv", index=False)
print("Saved: lysosome_blobs_regions_ALL.csv (all) and lysosome_blobs_regions.csv (filtered)")

ALLOW_BLOB_FALLBACK = (len(df_all) == 0)

# ==========================================
# CH2 segmentation (neurites mask)
# ==========================================
vol = img_ch2.astype(np.float32)
vmin, vmax = float(vol.min()), float(vol.max())
if vmax > vmin:
    vol = (vol - vmin) / (vmax - vmin)
else:
    vol[:] = 0.0

ch2 = gaussian(vol, sigma=CH2_SMOOTH_SIGMA, preserve_range=True)

neuron_mask = np.zeros_like(ch2, dtype=bool)
for z in range(ch2.shape[0]):
    R = ch2[z]
    t = threshold_local(R, block_size=THRESH_BLOCK_SIZE, offset=-THRESH_OFFSET_STD_MULT * np.std(R))
    neuron_mask[z] = R > t

if NEURITE_MODE:
    closing_radius_um = float(CLOSING_RADIUS_VOX) * float(np.sqrt(vx_um * vy_um))
    fp_close = ellipsoid_footprint_from_radius_um(closing_radius_um, vz_um, vy_um, vx_um)

    neuron_mask = binary_closing(neuron_mask, footprint=fp_close)
    neuron_mask = remove_small_objects(neuron_mask, min_size=30, connectivity=3)

    # ---- FIXED CALL (cos_min is now accepted) ----
    neuron_mask = stitch_neurite_fragments_by_orientation(
        neuron_mask,
        vx_um=vx_um, vy_um=vy_um, vz_um=vz_um,
        max_gap_um=float(MAX_GAP_UM),
        cos_min=float(cfg["STITCH_COS_MIN"]),
        bridge_dilate_radius_vox=3,
        intensity_img=ch2,
        intensity_min_frac=float(cfg["STITCH_INTENSITY_MIN_FRAC"]),
    )

    neuron_mask = remove_small_objects(neuron_mask, min_size=200, connectivity=3)
else:
    neuron_mask = binary_fill_holes(neuron_mask)

print("neurite voxels:", int(neuron_mask.sum()))

# Export AUTO mask to allow manual editing
tiff.imwrite("neuron_mask_AUTO.tif", (neuron_mask.astype(np.uint8) * 255))
print("Saved: neuron_mask_AUTO.tif (edit in Napari, save as neuron_mask_edited.tif if needed)")

# ---- ID segmentation ----
if NEURITE_MODE:
    if USE_EDITED_MASK_IF_EXISTS and os.path.isfile(EDITED_MASK_PATH):
        m = tiff.imread(EDITED_MASK_PATH)
        m = np.squeeze(m)
        if m.shape != neuron_mask.shape:
            raise ValueError(f"Edited mask shape {m.shape} != expected {neuron_mask.shape}")
        neuron_mask = (m > 0)
        print(f"Using edited mask: {EDITED_MASK_PATH}")

    cell_seg = label(neuron_mask, connectivity=3).astype(np.int32)
    cell_mask = neuron_mask.copy()
    print("Detected components (neurite networks):", int(cell_seg.max()))
else:
    dist = _edt(neuron_mask)
    cell_min_radius_vox = 1#########################################################################################################################
    cell_mask = (dist >= cell_min_radius_vox)
    cell_mask &= neuron_mask

    body_lab = label(cell_mask, connectivity=3)
    n_cells = int(body_lab.max())
    print(f"Detected {n_cells} cells (soma).")
    if n_cells > 0:
        dist_inside = _edt(neuron_mask)
        from skimage.segmentation import watershed
        cell_seg = watershed(-dist_inside, markers=body_lab, mask=neuron_mask)
    else:
        cell_seg = np.zeros_like(neuron_mask, dtype=np.int32)

print("ID voxels:", int((cell_seg > 0).sum()))

# ==========================================
# Visualization-only filtering (hide tiny components) + serial IDs
# ==========================================
cell_seg_viz = cell_seg.copy()
cell_id_map_viz = {}

if isinstance(cell_seg_viz, np.ndarray) and cell_seg_viz.max() > 0:
    counts_viz = np.bincount(cell_seg_viz.ravel().astype(np.int64))
    tiny_labels = np.where(counts_viz < VIZ_MIN_VOXELS)[0]
    tiny_labels = tiny_labels[tiny_labels > 0]
    if tiny_labels.size > 0:
        cell_seg_viz[np.isin(cell_seg_viz, tiny_labels)] = 0

    unique_labels = np.unique(cell_seg_viz)
    unique_labels = unique_labels[unique_labels > 0]
    if unique_labels.size > 0:
        new_seg = np.zeros_like(cell_seg_viz, dtype=np.int32)
        for new_id, old_id in enumerate(unique_labels, start=1):
            new_seg[cell_seg_viz == old_id] = new_id
            cell_id_map_viz[old_id] = new_id
        cell_seg_viz = new_seg

cell_mask_viz = (cell_seg_viz > 0)

# ==========================================
# Classify lysosomes: inside vs outside + assign ID
# ==========================================
dist_out_um = _edt(~neuron_mask, sampling=(vz_um, vy_um, vx_um)).astype(np.float32)
soft_cell_mask = neuron_mask | (dist_out_um <= MARGIN_UM)

location_ch2 = []
cell_id_list = []

if len(df) > 0:
    Z, Y, X = neuron_mask.shape
    for (zc_um, yc_um, xc_um, r_um) in df[["z_um", "y_um", "x_um", "radius_um"]].to_numpy():
        zz = int(round(zc_um / vz_um))
        yy = int(round(yc_um / vy_um))
        xx = int(round(xc_um / vx_um))

        inside_hard = (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X and neuron_mask[zz, yy, xx])
        inside_soft = (0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X and soft_cell_mask[zz, yy, xx])

        is_inside = bool(inside_hard)
        if (not is_inside) and inside_soft:
            is_inside = True

        if not is_inside:
            frac = sphere_overlap_fraction(zc_um, yc_um, xc_um, r_um, neuron_mask, vx_um, vy_um, vz_um)
            if frac >= OVERLAP_ALPHA:
                is_inside = True

        if is_inside:
            cid = 0
            if 0 <= zz < Z and 0 <= yy < Y and 0 <= xx < X:
                if cell_seg[zz, yy, xx] != 0:
                    cid = int(cell_seg[zz, yy, xx])
                else:
                    cid = nearest_cell_label(cell_seg, zz, yy, xx, max_r=NEIGHBOR_MAX_VOX)
            location_ch2.append("cell")
            cell_id_list.append(cid)
        else:
            location_ch2.append("outside")
            cell_id_list.append(0)

    df["location_ch2"] = location_ch2
    df["cell_id_ch2"] = cell_id_list
    df["cell_id_ch2_viz"] = df["cell_id_ch2"].map(cell_id_map_viz).fillna(0).astype(int) if isinstance(cell_id_map_viz, dict) else 0

    df["lys_id_in_cell"] = 0
    mask_in = (df["location_ch2"] == "cell") & (df["cell_id_ch2"] > 0)
    df_sorted = df.loc[mask_in].sort_values(["cell_id_ch2", "z_um", "y_um", "x_um"]).copy()
    df.loc[df_sorted.index, "lys_id_in_cell"] = (df_sorted.groupby("cell_id_ch2").cumcount().to_numpy() + 1).astype(int)

    df.to_csv("lysosomes_with_cell_vs_outside.csv", index=False)
    print("Saved: lysosomes_with_cell_vs_outside.csv (filtered)")

# ==========================================
# Per-ID volumes (µm^3)
# ==========================================
if isinstance(cell_seg, np.ndarray) and cell_seg.max() > 0:
    counts = np.bincount(cell_seg.ravel().astype(np.int64))
    cell_ids = np.arange(1, counts.size, dtype=int)
    voxels = counts[1:].astype(np.int64)
    vol_um3 = voxels.astype(float) * voxel_um3
    cell_volume_df = pd.DataFrame({"cell_id_ch2": cell_ids, "voxel_count": voxels, "volume_um3": vol_um3})
    cell_volume_df.to_csv("cell_volumes_ch2.csv", index=False)
    print("Saved: cell_volumes_ch2.csv")

# ==========================================
# Export overlay
# ==========================================
def _interval_tag(dmin, dmax):
    dmin = float(dmin)
    if dmax is None:
        return f"diam_{dmin:g}_to_inf_um"
    return f"diam_{dmin:g}_to_{float(dmax):g}_um"

INTERVAL_TAG = _interval_tag(DIAMETER_MIN_UM, DIAMETER_MAX_UM)

if EXPORT_FULLSIZE_OVERLAY:
    export_fullsize_overlay_stack(
        img_ch1=img_ch1,
        img_ch2_raw=img_ch2,
        cell_seg_viz=cell_seg_viz,
        df=df,
        vx_um=vx_um, vy_um=vy_um, vz_um=vz_um,
        output_dir=output_dir,
        alpha_labels=0.45,
        draw_only_inside=True,
        fps=FPS,
        basename=f"FULLSIZE_overlay_ID_Lysosomes_MAGENTA_{INTERVAL_TAG}",
    )

# ==========================================
# Simple fused video (optional)
# ==========================================
if GENERATE_VIDEOS:
    img_norm_2 = (ch2 * 255).astype(np.uint8)
    frames = []
    Z = img_norm_2.shape[0]
    px_um_xy = float(np.sqrt(vx_um * vy_um))

    for z in range(Z):
        base = cv2.cvtColor(img_norm_2[z], cv2.COLOR_GRAY2BGR)

        mask_u8 = (cell_mask_viz[z].astype(np.uint8) * 255)
        overlay = base.copy()
        overlay[..., 1] = np.maximum(overlay[..., 1], mask_u8)
        overlay = cv2.addWeighted(base, 1.0, overlay, 0.35, 0.0)

        drew_any = False
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            dfv = df[np.isfinite(df["z_um"]) & np.isfinite(df["y_um"]) & np.isfinite(df["x_um"]) & np.isfinite(df["radius_um"])]
            if not dfv.empty:
                zc = (dfv["z_um"].to_numpy() / vz_um).astype(float)
                yc = (dfv["y_um"].to_numpy() / vy_um).astype(float)
                xc = (dfv["x_um"].to_numpy() / vx_um).astype(float)
                r_um = dfv["radius_um"].to_numpy().astype(float)

                dz_um = np.abs(zc - z) * vz_um
                hits = dz_um <= r_um
                if np.any(hits):
                    r_proj_um = np.sqrt(np.clip(r_um[hits]**2 - dz_um[hits]**2, 0.0, None))
                    r_proj_px = r_proj_um / max(px_um_xy, 1e-12)
                    ys = np.rint(yc[hits]).astype(int)
                    xs = np.rint(xc[hits]).astype(int)

                    H, W = cell_mask_viz.shape[1], cell_mask_viz.shape[2]
                    for y, x, rpv in zip(ys, xs, r_proj_px):
                        rr = int(max(3, round(rpv)))
                        if 0 <= y < H and 0 <= x < W and rr > 0:
                            cv2.circle(overlay, (x, y), rr, LYS_EDGE_BGR, 4, lineType=cv2.LINE_AA)
                            cv2.circle(overlay, (x, y), rr, LYS_MAGENTA_BGR, 2, lineType=cv2.LINE_AA)
                            cv2.circle(overlay, (x, y), 1,  LYS_MAGENTA_BGR, -1, lineType=cv2.LINE_AA)
                    drew_any = True

        if (not drew_any) and ALLOW_BLOB_FALLBACK and (blobs is not None) and (len(blobs) > 0):
            z_blobs = blobs[np.abs(blobs[:, 0] - z) < 0.5]
            H, W = cell_mask_viz.shape[1], cell_mask_viz.shape[2]
            for b in z_blobs:
                y, x = int(round(b[1])), int(round(b[2]))
                r = int(max(3, round(b[3])))
                if 0 <= y < H and 0 <= x < W and r > 0:
                    cv2.circle(overlay, (x, y), r, LYS_EDGE_BGR, 4, lineType=cv2.LINE_AA)
                    cv2.circle(overlay, (x, y), r, LYS_MAGENTA_BGR, 2, lineType=cv2.LINE_AA)
                    cv2.circle(overlay, (x, y), 1, LYS_MAGENTA_BGR, -1, lineType=cv2.LINE_AA)

        cv2.putText(
            overlay, f"Mask + Lysosomes [{INTERVAL_TAG}]",
            (10, 22), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA
        )
        frames.append(overlay)

    try:
        imageio.mimsave(f"ch2_fused_cell_magenta_{INTERVAL_TAG}.mp4", frames, fps=int(FPS), format="FFMPEG")
        print(f"Saved: ch2_fused_cell_magenta_{INTERVAL_TAG}.mp4")
    except TypeError:
        imageio.mimsave(f"ch2_fused_cell_magenta_{INTERVAL_TAG}.gif", frames, fps=int(FPS))
        print(f"Saved: ch2_fused_cell_magenta_{INTERVAL_TAG}.gif")

# ==========================================
# Napari visualization + editing + filtering
# ==========================================
if LAUNCH_VIEWER:
    viewer = napari.Viewer()
    viewer.dims.ndisplay = 3

    viewer.add_image(img_ch2, name="Ch2 raw")
    viewer.add_image(img_ch1, name="Ch1 raw")

    mask_layer = viewer.add_labels(
        cell_mask_viz.astype(np.uint8),
        name="Neurite mask" if NEURITE_MODE else "Cell mask",
        opacity=0.35
    )
    try:
        mask_layer.blending = "translucent_no_depth"
    except Exception:
        pass

    n_labels = int(cell_seg_viz.max()) if isinstance(cell_seg_viz, np.ndarray) else 0
    cmap_u8 = make_label_colormap(n_labels, seed_hue=0.13)

    label_color = {0: (0.0, 0.0, 0.0, 0.0)}
    for i in range(1, n_labels + 1):
        r, g, b = cmap_u8[i].astype(np.float32) / 255.0
        label_color[i] = (float(r), float(g), float(b), 1.0)

    def _add_labels_with_color(data, name, opacity=0.25, visible=True):
        try:
            layer = viewer.add_labels(
                data, name=name, opacity=float(opacity), visible=bool(visible), color=label_color
            )
            return layer
        except TypeError:
            layer = viewer.add_labels(data, name=name, opacity=float(opacity), visible=bool(visible))
            try:
                layer.color = label_color
            except Exception:
                pass
            return layer

    id_layer = _add_labels_with_color(cell_seg_viz.astype(np.uint16), "ID (serial)", opacity=0.25, visible=True)
    try:
        id_layer.blending = "translucent_no_depth"
    except Exception:
        pass

    id_filtered_layer = _add_labels_with_color(
        np.zeros_like(cell_seg_viz, dtype=np.uint16),
        "ID (filtered view)",
        opacity=0.45,
        visible=False
    )
    try:
        id_filtered_layer.blending = "translucent_no_depth"
    except Exception:
        pass

    pts_layer = None
    view_pts_layer = None

    serial_to_original_id = None
    if isinstance(cell_id_map_viz, dict) and len(cell_id_map_viz) > 0:
        serial_to_original_id = {serial_id: original_id for (original_id, serial_id) in cell_id_map_viz.items()}

    def _points_rgba_from_props(props_dict):
        loc = np.array(props_dict.get("location_ch2", []), dtype=str)
        cid = np.array(props_dict.get("cell_id_serial", []), dtype=int)

        n = int(len(cid))
        rgba = np.zeros((n, 4), dtype=np.float32)
        rgba[:] = (1.0, 0.0, 1.0, 0.85)

        inside = (loc == "cell") & (cid > 0)
        if np.any(inside):
            for k in np.where(inside)[0]:
                c = int(cid[k])
                rgba[k] = np.array(label_color.get(c, (1.0, 0.0, 1.0, 0.85)), dtype=np.float32)
        return rgba

    def _refresh_point_colors(layer):
        if layer is None:
            return
        p = dict(layer.properties)
        try:
            layer.face_color = _points_rgba_from_props(p)
        except Exception:
            pass

    _filter_state = {
        "cell_ids": None,
        "location": "both",
        "lys_ids": None,
        "show_filtered_labels": True,
        "show_view_layer": True,
    }

    def _parse_id_text(s, max_id):
        s = (s or "").strip().lower()
        if s in ("", "all", "*"):
            return list(range(1, int(max_id) + 1))

        parts = re.split(r"[,\s;]+", s)
        out = set()
        for part in parts:
            part = part.strip()
            if not part:
                continue
            if "-" in part:
                a, b = part.split("-", 1)
                try:
                    a = int(a); b = int(b)
                except Exception:
                    continue
                lo, hi = (a, b) if a <= b else (b, a)
                for v in range(lo, hi + 1):
                    if 1 <= v <= int(max_id):
                        out.add(v)
            else:
                try:
                    v = int(part)
                except Exception:
                    continue
                if 1 <= v <= int(max_id):
                    out.add(v)
        return sorted(out)

    def _parse_lys_text(s):
        s = (s or "").strip().lower()
        if s in ("", "all", "*"):
            return None
        parts = re.split(r"[,\s;]+", s)
        out = set()
        for part in parts:
            part = part.strip()
            if not part:
                continue
            if "-" in part:
                a, b = part.split("-", 1)
                try:
                    a = int(a); b = int(b)
                except Exception:
                    continue
                lo, hi = (a, b) if a <= b else (b, a)
                for v in range(lo, hi + 1):
                    if v >= 1:
                        out.add(v)
            else:
                try:
                    v = int(part)
                except Exception:
                    continue
                if v >= 1:
                    out.add(v)
        return sorted(out) if len(out) else None

    def _update_filtered_labels(sel_ids):
        if sel_ids is None or len(sel_ids) == 0:
            id_filtered_layer.visible = False
            id_filtered_layer.data = np.zeros_like(cell_seg_viz, dtype=np.uint16)
            return
        mask = np.isin(cell_seg_viz, np.array(sel_ids, dtype=np.int32))
        out = np.zeros_like(cell_seg_viz, dtype=np.uint16)
        out[mask] = cell_seg_viz[mask].astype(np.uint16)
        id_filtered_layer.data = out
        id_filtered_layer.visible = True

    def _apply_points_filter(sel_ids, location_mode="both", lys_ids=None, show_view_layer=True):
        if pts_layer is None or view_pts_layer is None:
            return
        p = dict(pts_layer.properties)
        loc = np.array(p.get("location_ch2", []), dtype=str)
        cid = np.array(p.get("cell_id_serial", []), dtype=int)
        lys = np.array(p.get("lys_id_serial", []), dtype=int)

        sel_ids = set(int(x) for x in (sel_ids or []))
        keep = np.isin(cid, list(sel_ids))

        if location_mode == "cell":
            keep &= (loc == "cell")
        elif location_mode == "outside":
            keep &= (loc != "cell")

        if lys_ids is not None:
            lys_ids = set(int(x) for x in lys_ids)
            keep &= np.isin(lys, list(lys_ids))

        idx = np.where(keep)[0]

        view_pts_layer.data = np.asarray(pts_layer.data)[idx]
        view_pts_layer.size = np.asarray(pts_layer.size)[idx]
        view_props = {k: np.asarray(v)[idx] for k, v in p.items()}
        view_pts_layer.properties = view_props
        _refresh_point_colors(view_pts_layer)
        view_pts_layer.visible = bool(show_view_layer)

    def _reapply_current_filter():
        if _filter_state["cell_ids"] is None:
            return
        if _filter_state["show_filtered_labels"]:
            _update_filtered_labels(_filter_state["cell_ids"])
        else:
            _update_filtered_labels([])
        _apply_points_filter(
            _filter_state["cell_ids"],
            location_mode=_filter_state["location"],
            lys_ids=_filter_state["lys_ids"],
            show_view_layer=_filter_state["show_view_layer"],
        )

    # ---- Build points layer if requested ----
    if EDIT_LYSOSOME_TABLE_IN_NAPARI and isinstance(df, pd.DataFrame) and len(df) > 0:
        df_edit = attach_all_blob_fields(df.copy())

        pts_zyx = np.stack([
            (df_edit["z_um"].to_numpy() / vz_um),
            (df_edit["y_um"].to_numpy() / vy_um),
            (df_edit["x_um"].to_numpy() / vx_um),
        ], axis=1).astype(np.float32)

        if "radius_um" in df_edit.columns:
            radii_vox = df_edit["radius_um"].to_numpy(dtype=float) / (np.sqrt(vx_um * vy_um) + 1e-12)
            sizes = np.clip(radii_vox * 2, 2, None).astype(np.float32)
        else:
            sizes = np.full((pts_zyx.shape[0],), 6, dtype=np.float32)

        if "location_ch2" not in df_edit.columns:
            df_edit["location_ch2"] = "outside"

        if "cell_id_serial" in df_edit.columns:
            df_edit["cell_id_serial"] = df_edit["cell_id_serial"].fillna(0).astype(int)
        elif "cell_id_ch2_viz" in df_edit.columns:
            df_edit["cell_id_serial"] = df_edit["cell_id_ch2_viz"].fillna(0).astype(int)
        else:
            df_edit["cell_id_serial"] = df_edit["cell_id_ch2"].map(cell_id_map_viz).fillna(0).astype(int)

        if "diameter_um" not in df_edit.columns:
            df_edit["diameter_um"] = np.nan
        if "peak_gray" not in df_edit.columns:
            df_edit["peak_gray"] = np.nan

        def _recompute_lys_id_serial(cell_id_serial_arr, location_arr):
            n = int(len(cell_id_serial_arr))
            out = np.zeros(n, dtype=int)
            mask = (location_arr.astype(str) == "cell") & (cell_id_serial_arr.astype(int) > 0)
            if not np.any(mask):
                return out
            tmp = pd.DataFrame({
                "idx": np.arange(n, dtype=int),
                "cell_id_serial": cell_id_serial_arr.astype(int),
                "z_um": df_edit["z_um"].to_numpy(dtype=float),
                "y_um": df_edit["y_um"].to_numpy(dtype=float),
                "x_um": df_edit["x_um"].to_numpy(dtype=float),
            })
            tmp = tmp.loc[mask].sort_values(["cell_id_serial", "z_um", "y_um", "x_um"], kind="mergesort")
            serial = (tmp.groupby("cell_id_serial").cumcount().to_numpy() + 1).astype(int)
            out[tmp["idx"].to_numpy(dtype=int)] = serial
            return out

        loc0 = df_edit["location_ch2"].astype(str).to_numpy()
        cell_serial0 = df_edit["cell_id_serial"].astype(int).to_numpy()
        lys_serial0 = _recompute_lys_id_serial(cell_serial0, loc0)
        df_edit["lys_id_serial"] = lys_serial0

        props = {
            "location_ch2": loc0,
            "cell_id_serial": cell_serial0,
            "lys_id_serial": lys_serial0,
            "diameter_um": df_edit["diameter_um"].to_numpy(dtype=float),
            "peak_gray": df_edit["peak_gray"].to_numpy(dtype=float),
        }

        pts_layer = viewer.add_points(
            pts_zyx,
            size=sizes,
            name="Lysosomes (EDIT TABLE)",
            properties=props,
        )
        pts_layer.edge_color = "black"
        pts_layer.edge_width = 0.3
        pts_layer.mode = "select"
        pts_layer.text = {"string": "{cell_id_serial}:{lys_id_serial}", "size": 10, "color": "white"}
        _refresh_point_colors(pts_layer)

        view_pts_layer = viewer.add_points(
            pts_zyx[:0],
            size=np.array([], dtype=np.float32),
            name="Lysosomes (VIEW FILTER)",
            properties={k: np.asarray(v)[:0] for k, v in props.items()},
        )
        view_pts_layer.edge_color = "black"
        view_pts_layer.edge_width = 0.2
        view_pts_layer.mode = "pan_zoom"
        view_pts_layer.visible = False
        view_pts_layer.text = {"string": "{cell_id_serial}:{lys_id_serial}", "size": 5, "color": "white"}

        def _refresh_labels_and_serials():
            if pts_layer is None:
                return
            p = dict(pts_layer.properties)
            loc = np.array(p["location_ch2"]).astype(str)
            cell_serial = np.array(p["cell_id_serial"]).astype(int)
            lys_serial = _recompute_lys_id_serial(cell_serial, loc)
            p["lys_id_serial"] = lys_serial
            pts_layer.properties = p
            _refresh_point_colors(pts_layer)
            _reapply_current_filter()

        def _apply_props_update(indices, new_loc=None, new_cell_serial=None):
            if pts_layer is None or not indices:
                return
            p = dict(pts_layer.properties)
            loc = np.array(p["location_ch2"]).astype(object)
            cell_serial = np.array(p["cell_id_serial"]).astype(int)

            for i in indices:
                if new_loc is not None:
                    loc[i] = str(new_loc)
                if new_cell_serial is not None:
                    sid = int(new_cell_serial)
                    cell_serial[i] = sid
                if new_loc == "outside":
                    cell_serial[i] = 0

            p["location_ch2"] = loc.astype(str)
            p["cell_id_serial"] = cell_serial.astype(int)
            pts_layer.properties = p
            _refresh_labels_and_serials()

        @viewer.bind_key("A")
        def assign_selected_to_id_under_cursor(event=None):
            if pts_layer is None:
                return
            sel = sorted(list(pts_layer.selected_data))
            if not sel:
                print("[Napari edit] No points selected.")
                return

            zf, yf, xf = viewer.cursor.position
            zz, yy, xx = int(round(zf)), int(round(yf)), int(round(xf))
            if not (0 <= zz < cell_seg_viz.shape[0] and 0 <= yy < cell_seg_viz.shape[1] and 0 <= xx < cell_seg_viz.shape[2]):
                print("[Napari edit] Cursor out of bounds.")
                return

            serial_id = int(cell_seg_viz[zz, yy, xx])
            if serial_id <= 0:
                print("[Napari edit] Cursor not over a labeled SERIAL ID.")
                return

            _apply_props_update(sel, new_loc="cell", new_cell_serial=serial_id)
            print(f"[Napari edit] Assigned {len(sel)} lysosomes -> serial ID {serial_id}")

        @viewer.bind_key("X")
        def mark_selected_outside(event=None):
            if pts_layer is None:
                return
            sel = sorted(list(pts_layer.selected_data))
            if not sel:
                print("[Napari edit] No points selected.")
                return
            _apply_props_update(sel, new_loc="outside", new_cell_serial=0)
            print(f"[Napari edit] Marked {len(sel)} lysosomes as outside")

        def _export_edited_csv(path):
            if pts_layer is None:
                return
            p = pts_layer.properties
            df_out = df_edit.copy()
            df_out["location_ch2"] = np.array(p["location_ch2"]).astype(str)
            df_out["cell_id_serial"] = np.array(p["cell_id_serial"]).astype(int)
            df_out["lys_id_serial"] = np.array(p["lys_id_serial"]).astype(int)
            df_out = attach_all_blob_fields(df_out)
            df_out.to_csv(path, index=False)
            print(f"[Napari edit] Saved edited lysosome table: {path}")

        @viewer.bind_key("S")
        def save_now(event=None):
            _export_edited_csv(LYSOSOME_EDITED_CSV)

        try:
            from magicgui import magicgui

            @magicgui(
                call_button="Apply filter",
                ids_text={"label": "Cell IDs (e.g. all, 1,2,5-8)", "value": "all"},
                lys_text={"label": "Lys IDs in those cells (e.g. all, 1,2 or 1-3)", "value": "all"},
                location={"choices": ["both", "cell", "outside"], "value": "both"},
                show_filtered_labels={"label": "Show filtered ID labels layer", "value": True},
                show_view_layer={"label": "Show VIEW FILTER points layer", "value": True},
            )
            def filter_panel(ids_text="all", lys_text="all", location="both", show_filtered_labels=True, show_view_layer=True):
                sel_ids = _parse_id_text(ids_text, n_labels)
                lys_ids = _parse_lys_text(lys_text)

                _filter_state["cell_ids"] = sel_ids
                _filter_state["location"] = str(location)
                _filter_state["lys_ids"] = lys_ids
                _filter_state["show_filtered_labels"] = bool(show_filtered_labels)
                _filter_state["show_view_layer"] = bool(show_view_layer)

                _reapply_current_filter()
                msg_lys = "all" if lys_ids is None else str(lys_ids)
                print(f"[Filter] cells={sel_ids} location={location} lys={msg_lys}")

            @magicgui(call_button="Clear filter")
            def clear_filter_panel():
                _filter_state["cell_ids"] = None
                _filter_state["location"] = "both"
                _filter_state["lys_ids"] = None
                _filter_state["show_filtered_labels"] = True
                _filter_state["show_view_layer"] = True

                _update_filtered_labels([])
                if view_pts_layer is not None:
                    view_pts_layer.visible = False
                    view_pts_layer.data = np.zeros((0, 3), dtype=np.float32)
                print("[Filter] Cleared.")

            viewer.window.add_dock_widget(filter_panel, area="right", name="Filter cells / lysosomes")
            viewer.window.add_dock_widget(clear_filter_panel, area="right", name="Clear filter")

        except Exception as e:
            print("[Napari] magicgui not available; filter panel disabled. Error:", e)

    try:
        viewer.camera.zoom = 1.2
    except Exception:
        pass

    napari.run()

    # Auto-save on close
    if EDIT_LYSOSOME_TABLE_IN_NAPARI and pts_layer is not None:
        try:
            p = pts_layer.properties
            df_out = df_edit.copy()
            df_out["location_ch2"] = np.array(p["location_ch2"]).astype(str)
            df_out["cell_id_serial"] = np.array(p["cell_id_serial"]).astype(int)
            df_out["lys_id_serial"] = np.array(p["lys_id_serial"]).astype(int)
            df_out = attach_all_blob_fields(df_out)
            df_out.to_csv(LYSOSOME_EDITED_CSV, index=False)
            print(f"[Napari edit] Auto-saved edited lysosome table: {LYSOSOME_EDITED_CSV}")
        except Exception as e:
            print("[Napari edit] Auto-save failed:", e)